The purpose of this notebook is to compute reach statistics based on retrospective model predictions.

**Requirements**  
- hsclient  
- S3hsclient
- pandas
- pyarrow
- s3fs  

In [3]:
import time
import pandas
import S3hsclient as hsclient
import matplotlib.pyplot as plt

Historical streamflow data has been collected and stored in HydroShare at: [https://www.hydroshare.org/resource/446999d3f27149cea4ee9863d2de7ad2/](https://www.hydroshare.org/resource/446999d3f27149cea4ee9863d2de7ad2/) . We can access these data via HydroShare's S3 integration.

In [4]:
hs = hsclient.S3HydroShare()

Username:  TonyCastronova
Password for TonyCastronova:  ········


In [5]:
res = hs.resource('446999d3f27149cea4ee9863d2de7ad2')

In [6]:
res.s3_ls()

['tonycastronova/446999d3f27149cea4ee9863d2de7ad2/data/contents/DeSoto.parquet',
 'tonycastronova/446999d3f27149cea4ee9863d2de7ad2/data/contents/MountAscutney.parquet',
 'tonycastronova/446999d3f27149cea4ee9863d2de7ad2/data/contents/RoaringRiver.parquet',
 'tonycastronova/446999d3f27149cea4ee9863d2de7ad2/data/contents/SpringfieldGreeneCounty.parquet',
 'tonycastronova/446999d3f27149cea4ee9863d2de7ad2/data/contents/TwoRiversOttauquechee.parquet',
 'tonycastronova/446999d3f27149cea4ee9863d2de7ad2/data/contents/Windham.parquet']

In [ ]:
df = pandas.read_parquet('tonycastronova/446999d3f27149cea4ee9863d2de7ad2/data/contents/DeSoto.parquet', engine='pyarrow', filesystem=res._hs_session.s3)

Compute quantiles for all reaches.

In [ ]:
for region_name in ['DeSoto',
                    'MountAscutney',
                    'RoaringRiver',
                    'SpringfieldGreeneCounty',
                    'TwoRiversOttauquechee',
                    'Windham'
                    ]:
    st = time.time()
    print(f'Processing {region_name}...', end='', flush=True)
    df = pandas.read_parquet(f'tonycastronova/446999d3f27149cea4ee9863d2de7ad2/data/contents/{region_name}.parquet',
                             engine='pyarrow',
                             filesystem=res._hs_session.s3)
    
    quantile_levels = [0, 0.5, 0.10, 0.25, 0.75, 0.90, 1.0]
    quantile_names = ['q0', 'q5', 'q10', 'q25', 'q75', 'q90', 'q100']
    
    # make sure 'doy' exists
    df['doy'] = df['time'].dt.dayofyear
    
    # compute quantiles grouped by feature_id and doy
    quantiles = (
        df.groupby(['feature_id', 'doy'])['streamflow']
          .quantile(quantile_levels)
          .unstack()  # quantiles become columns
          .rename(columns=dict(zip(quantile_levels, quantile_names)))
          .reset_index()
    )
    
    quantiles.to_parquet(f'{region_name}_quantiles.parquet')

    print(f'done. [{time.time() - st} sec elapsed]')
    

Processing TwoRiversOttauquechee...

Speed this up using dask (this isn't faster on my local computer, probably need more resources)

In [5]:
import dask.dataframe as dd
from dask.distributed import Client

In [17]:
# use a try accept loop so we only instantiate the client
# if it doesn't already exist.
try:
    print(client.dashboard_link)
except:    
    # The client should be customized to your workstation resources.
    # This is configured for a "Large" instance on ciroh.awi.2i2c.cloud
    # client = Client()
    client = Client(
                    n_workers=2,
                    memory_limit='8GB',
                    local_directory="./dask-worker-space" # where to spill data 
    )
    print(client.dashboard_link)

http://127.0.0.1:8787/status


CPU times: user 85.9 ms, sys: 27.7 ms, total: 114 ms
Wall time: 758 ms


,time,feature_id,streamflow,doy
npartitions=3,,,,
,datetime64[ns],int64,float64,int32
,...,...,...,...
,...,...,...,...
,...,...,...,...


In [ ]:
%%time

region = 'DeSoto'

ddf = dd.read_parquet(f'tonycastronova/446999d3f27149cea4ee9863d2de7ad2/data/contents/{region}.parquet',
                      filesystem=res._hs_session.s3)
ddf['doy'] = ddf['time'].dt.dayofyear
ddf

N_CHUNKS = 8   

# Get sorted unique feature_ids
feature_ids = ddf['feature_id'].unique().compute()
feature_ids = np.sort(feature_ids)

# Split feature_ids into N_CHUNKS
chunk_size = math.ceil(len(feature_ids) / N_CHUNKS)
feature_id_chunks = [feature_ids[i:i+chunk_size] for i in range(0, len(feature_ids), chunk_size)]

# Folder for temporary parquet files
tmp_dir = "tmp_quantiles"
os.makedirs(tmp_dir, exist_ok=True)


quantile_levels = [0.0, 0.5, 0.10, 0.25, 0.75, 0.90, 1.0]
quantile_names = ['q0', 'q5', 'q10', 'q25', 'q75', 'q90', 'q100']

def compute_quantiles(subdf):
    q = subdf['streamflow'].quantile(quantile_levels)
    q.index = quantile_names
    return pandas.Series(q)

# Process each chunk
output_files = []
for i, ids_chunk in enumerate(feature_id_chunks):
    print(f"Processing chunk {i+1}/{len(feature_id_chunks)} with {len(ids_chunk)} feature_ids...")

    ddf_chunk = ddf[ddf['feature_id'].isin(ids_chunk)]
    quantiles_chunk = (
        ddf_chunk.groupby(['feature_id', 'doy'])
                 .apply(compute_quantiles,
                        meta={name: 'f8' for name in quantile_names},
                        include_groups=False)
    ).compute()

    quantiles_chunk.reset_index(inplace=True)
    quantiles_chunk.columns.name = None

    outfile = os.path.join(tmp_dir, f"quantiles_part_{i}.parquet")
    quantiles_chunk.to_parquet(outfile) #, write_index=False)
    output_files.append(outfile)

# Merge all chunks into a final file
print("Merging all parquet chunks...")
ddf_merged = dd.read_parquet(output_files)
df = ddf_merged.reset_index().compute

final_file = f"{region}_quantiles.parquet"
df.to_parquet(final_file, engine='pyarrow')

## Detect Good and Bad Computations

In [32]:
def detect_quantile_collapse(d, threshold=0.01):
    d.set_index('doy', inplace=True)
    diff = d.q75 - d.q25
    diff.loc[diff < threshold] = 1
    diff.loc[diff != 1] = 0
    return pandas.Series({'collapsed': sum(diff)})
    
regions = ['DeSoto', 'RoaringRiver', 'SpringfieldGreeneCounty', 'Windham', 'MountAscutney', 'TwoRiversOttauquechee']
dfs = []
for region in regions:
    df = pandas.read_parquet(f'{region}_quantiles.parquet', engine='pyarrow')
    
    # compute collapse scores
    scores = df.groupby('feature_id').apply(detect_quantile_collapse, include_groups=False)
    scores.reset_index(inplace=True)
    
    # drop reaches with too many collapses
    fids = scores[scores.collapsed <= 50].feature_id.dropna()
    good_data = df.where(df.feature_id.isin(fids)).dropna()

    dfs.append(good_data)

In [40]:
for d in dfs:
    print(len(d[d.feature_id == 9328724]))

0
0
0
0
366
366


In [53]:
# merge data frames and save the result to parquet
all_data = pandas.concat(dfs, ignore_index=True).drop(columns=['index'])
all_data.feature_id = all_data.feature_id.astype(int)

# drop any duplicates that might exist. This could happen if region boundaries 
# overlap slightly. A unique record is determined by combining 'feature_id' and 'doy'
all_data.drop_duplicates(subset=['feature_id', 'doy'], keep='first', inplace=True)


In [56]:
# drop any reach that contains NaNs
null_mask = all_data.isnull().any(axis=1)
nulls = all_data[null_mask]
null_reaches = nulls.feature_id.unique()
print(f'Found {len(null_reaches)} reaches containing NaN records')

if len(null_reaches) > 0:
    print('Dropping Reaches containing NaN')
    all_data = all_data[~all_data.feature_id.isin(null_reaches)]

Found 0 reaches containing NaN records


In [57]:
all_data.to_parquet(f'all_quantiles_good.parquet')